<a href="https://colab.research.google.com/github/arkalim/PyTorch/blob/master/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Define the transformations

In [2]:
# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

26427392it [00:00, 67678908.28it/s]                              


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 416243.28it/s]
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:00, 17398902.03it/s]                             
8192it [00:00, 152119.05it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


## Create data loaders to shuffle and create batches

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [4]:
print('No. of train images: {}'.format(len(trainset)))
print('No. of test images: {}'.format(len(testset)))

print('No. of train batches: {}'.format(len(trainloader)))
print('No. of test batches: {}'.format(len(testloader)))

No. of train images: 60000
No. of test images: 10000
No. of train batches: 60
No. of test batches: 100


## Create the Network

In [5]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
    
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(12 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.reshape(-1, 12 * 4 * 4)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        return x

# create an instance of the Network    
network = Network()

# move the network object to GPU
network.to(device)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

## Function to find the accuracy

In [0]:
def find_acc(pred, label):
    correct = pred.argmax(dim = 1).eq(label)
    accuracy = correct.to(torch.float32).mean().item()
    return accuracy

## Train the Network

In [7]:
num_epochs = 10

# loss defined using torch.nn
loss = nn.CrossEntropyLoss()

# define the optimizer
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)

# note the starting time to find the total time elapsed
start_time = time.time()

for epoch in range(1,num_epochs+1):
    
    loss_train = 0
    loss_valid = 0
    acc_train = 0
    acc_valid = 0
    
    for step in range(len(trainloader)):

        images , labels = next(iter(trainloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        labels = labels.to(device)
        
        pred = network(images)
        
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss for the current step
        loss_train_step = loss(pred , labels)
        
        # find accuracy
        acc_train_step = find_acc(pred, labels)
        
        # calculate the gradients
        loss_train_step.backward()
        
        # update the parameters
        optimizer.step()
        
        loss_train += loss_train_step.item()

        acc_train += acc_train_step        
        
    for step in range(len(testloader)):

        images , labels = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        labels = labels.to(device)
        
        pred = network(images)
        
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss for the current step
        loss_valid_step = loss(pred , labels)
        
        # find accuracy
        acc_valid_step = find_acc(pred, labels)
      
        loss_valid += loss_valid_step.item()
    
        acc_valid += acc_valid_step
    
    loss_train /= len(trainloader)
    loss_valid /= len(testloader)
   
    acc_train /= len(trainloader)
    acc_valid /= len(testloader)
    
    print('Epoch: {}  Train Loss: {}  Train Acc: {}  Valid Loss: {}  Valid Acc: {}'.format(epoch, loss_train, acc_train, loss_valid, acc_valid))
    
# find the time at the end of training    
end_time = time.time()

total_time = end_time - start_time
print("Total time taken : {}".format(total_time))

Epoch: 1  Train Loss: 2.3002537727355956  Train Acc: 0.11453333894411723  Valid Loss: 2.29201340675354  Valid Acc: 0.11999999731779099
Epoch: 2  Train Loss: 2.1909592787424725  Train Acc: 0.3203333499530951  Valid Loss: 1.6700690984725952  Valid Acc: 0.5
Epoch: 3  Train Loss: 1.0660977333784103  Train Acc: 0.5928500294685364  Valid Loss: 0.8286855816841125  Valid Acc: 0.6499999761581421
Epoch: 4  Train Loss: 0.8608739833037059  Train Acc: 0.6635000288486481  Valid Loss: 0.7773889899253845  Valid Acc: 0.6899999976158142
Epoch: 5  Train Loss: 0.7294968138138453  Train Acc: 0.7178833693265915  Valid Loss: 0.6904471516609192  Valid Acc: 0.7199999690055847
Epoch: 6  Train Loss: 0.6486187567313512  Train Acc: 0.7522000342607498  Valid Loss: 0.6682687997817993  Valid Acc: 0.7199999690055847
Epoch: 7  Train Loss: 0.61022962530454  Train Acc: 0.7698833684126536  Valid Loss: 0.6431833505630493  Valid Acc: 0.7199999690055847
Epoch: 8  Train Loss: 0.5745217581590016  Train Acc: 0.781450034181277  

## Testing the model

In [8]:
def test_model(model):

    start_time = time.time()

    num_correct = 0
    accuracy = 0

    # turning off backprop and gradient calculation.
    # this improves performance 
    with torch.no_grad():

        for batch in testloader:

            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            total_images = len(testset)

            pred = model(images)

            num_correct_batch = pred.argmax(dim = 1).eq(labels).sum().item()

            accuracy_batch = pred.argmax(dim = 1).eq(labels).float().mean().item()

            num_correct += num_correct_batch
            accuracy += accuracy_batch

        accuracy /= len(testloader)

    print('Total number of test images: {}'.format(total_images))
    print('Total number of correct predictions: {}'.format(num_correct))
    print('Accuracy: {}'.format(accuracy * 100))

    end_time = time.time()
    print("Elapsed Time : {}".format(end_time - start_time))
    
# test the trained network    
test_model(network)    

Total number of test images: 10000
Total number of correct predictions: 8003
Accuracy: 80.0299978852272
Elapsed Time : 0.8791663646697998


## Save and Restore

### Save and Load Model's Parameters
#### Saving the model's state_dict()

In [0]:
path = 'network_weights.pth'

# Save the parameters
torch.save(network.state_dict(), path)

#### Loading the model's state_dict()


In [10]:
# create a new model object 
new_network = Network()

# move the network to GPU
new_network.to(device)

# load the network's parameters
new_network.load_state_dict(torch.load(path))

# set the network into evaluate mode
new_network.eval()

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

#### Test

In [11]:
test_model(new_network)

Total number of test images: 10000
Total number of correct predictions: 8003
Accuracy: 80.0299978852272
Elapsed Time : 0.9047195911407471


## Saving and Loading entire Model

In [12]:
path = 'full_network.pth'

# save the model
torch.save(network, path)

# load the model
new_model = torch.load(path)
new_model.eval()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

#### Test

In [13]:
test_model(new_model)

Total number of test images: 10000
Total number of correct predictions: 8003
Accuracy: 80.0299978852272
Elapsed Time : 0.8959379196166992
